In [613]:
import pandas as pd
import numpy as np

In [614]:
cost_price_dict = {'63621/tab': 6.85, '64640/curve': 6.85, '63014/cb3': 6.85, 'cb-5': 6.85, '64638/cb-9': 6.85,
                   
                   'violet': 8.95, 'naomi': 8.95, 'audrey': 8.95, 'ava': 8.95, 'delilah': 8.95, 'rose': 8.95,
                   
                   '64295': 13.25, '64257': 20.90, '64296': 14.05, '62879': 14.00, '63931': 12.35, '64064': 15.75,
                   '63754': 13.00, '63753': 8.65, '64198': 16.65, '64203': 13.35, '63858': 18.00, '64033': 6.95,
                   
                   '31800': 2.95,
                   
                   'swansea': 6.95, 'perth': 6.95, 'ayr': 6.95, 'iverness': 7.95, 'meath': 10.75, 'bray': 10.75,
                   'mourne': 10.25, 'cardiff': 8.35, 'derby': 10.50, 'newport': 10.95, 'bristol': 7.50, 'london': 10.95,
                   'oxford': 10.95, 'york': 7.5, 'westminster': 10.95,

                   '64747': 18.50, '64739': 16.20, '64731': 16.00, '64741': 10.55, '64454': 15.50, '64732': 10.30,
                   '64787': 14.40, '64668': 18.35, '64867': 16.60, '64179': 12.60, '64562': 16.00, '64416': 15.20,
                   '64434': 16.55, '32049': 5.00, '32056': 3.25, '32057': 4.10, '32047': 4.10,}

In [615]:
df = pd.read_csv('transaction-logs (26).csv', dtype=str, delimiter=';')

temp2_df = df['Date created'].str.split('-', expand=True)
temp2_df.columns = ['f_', 's_']
temp2_df['f_'] = temp2_df['f_'].str.replace('...', '')
temp2_df['f_'] = temp2_df['f_'].str.lstrip()
temp2_df['f_'] = temp2_df['f_'].str.rstrip()
temp2_df['s_'] = temp2_df['s_'].str.lstrip()
temp2_df['s_'] = temp2_df['s_'].str.rstrip()
df['date'] = temp2_df['f_']
df['time'] = temp2_df['s_']
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
# df['date'] = df['date'].dt.date
df['date'] = df['date'].astype(str)

df = df[df['Type'] == 'Order amount']
df = df[df['Offer SKU'].str.contains('00')]
df['Quantity'] = df['Quantity'].astype(int)
df['Amount'] = df['Amount'].astype(float)
df.dropna(subset=['Offer SKU'], inplace=True)

temp2_df = df['date'].str.split('-', expand=True)
df['Year-Month'] = temp2_df[0] + '-' + temp2_df[1]

df.head(2)

,Date created,Date received,Transaction Date,Shop,Order number,Invoice number,Transaction Number,Quantity,Category Label,Offer SKU,...,Customer order reference,Shop order reference,Billing cycle date,Shop ID,Order line ID,Refund ID,Sales channel,date,time,Year-Month
116,06/02/2023 - 22:27:42,24/02/2023 - 00:22:00,NaN,Ashwood Leather,DUX009042255-A,000000233808,NaN,1,Womens Bags & Purses,0004524800,...,NaN,NaN,27/02/2023 - 00:24:43,2472,DUX009042255-A-1,NaN,NaN,2023-02-06,22:27:42,2023-02
168,10/02/2023 - 13:33:04,01/03/2023 - 16:17:17,NaN,Ashwood Leather,DUX009092275-A,000000234020,NaN,1,Womens Bags & Purses,0004525400,...,NaN,NaN,13/03/2023 - 00:23:40,2472,DUX009092275-A-1,NaN,NaN,2023-02-10,13:33:04,2023-02


In [616]:
df_wide = df.pivot_table(
    index="Offer SKU",
    columns="Year-Month",
    values="Quantity",
    aggfunc="sum",  # sum in case multiple sales in the same month
    fill_value=0    # fill missing months with 0
)

# Ensure columns are sorted chronologically
df_wide = df_wide.reindex(sorted(df_wide.columns), axis=1)

# def restock_start_after_prev_double_zero(row):
#     sales = row.values
#     months = row.index

#     # All zeros → never in stock
#     if (sales == 0).all():
#         return None

#     # Find all starting indices of runs with at least two consecutive zeros
#     zero_runs = []
#     i = 0
#     n = len(sales)
#     while i < n - 1:
#         if sales[i] == 0 and sales[i+1] == 0:
#             start = i
#             j = i + 2
#             while j < n and sales[j] == 0:
#                 j += 1
#             zero_runs.append((start, j - 1))  # (start_idx, end_idx) inclusive
#             i = j  # skip this whole zero run
#         else:
#             i += 1

#     if not zero_runs:
#         # No 0-0 anywhere → always in stock within the window
#         return months[0]

#     # If the series ends with a zero run, ignore that *last* run (sell-out tail)
#     # and use the previous zero run (if it exists); otherwise use the last run.
#     last_run_start, last_run_end = zero_runs[-1]
#     ends_with_zero_run = last_run_end == n - 1

#     if ends_with_zero_run and len(zero_runs) >= 2:
#         # Use the second-to-last zero run
#         use_run_start, use_run_end = zero_runs[-2]
#     else:
#         # Use the last zero run
#         use_run_start, use_run_end = last_run_start, last_run_end

#     # First non-zero AFTER the chosen zero run
#     k = use_run_end + 1
#     while k < n and sales[k] == 0:
#         k += 1
#     return months[k] if k < n else None

import re
import numpy as np
import pandas as pd

def restock_after_prev_zero_run(row, min_zeros=3):
    """
    Return the first non-zero month immediately AFTER the chosen zero run.
    We choose the last zero-run of length >= min_zeros, BUT if the series
    ends with such a zero run (sell-out tail), we ignore that and use the
    previous zero-run instead (if any).
    Columns must be in chronological order (oldest -> latest).
    """
    sales  = row.to_numpy()
    months = row.index.to_numpy()
    n = len(sales)

    # All zeros -> never in stock
    if np.all(sales == 0):
        return None

    # Find all zero-runs with length >= min_zeros
    zero_runs = []
    i = 0
    while i < n:
        if sales[i] == 0:
            start = i
            j = i
            while j < n and sales[j] == 0:
                j += 1
            if (j - start) >= min_zeros:
                zero_runs.append((start, j - 1))  # inclusive [start, end]
            i = j
        else:
            i += 1

    if not zero_runs:
        # No qualifying zero runs -> treat as "always in stock" in window
        return months[0]

    # If the last qualifying zero-run reaches the end, ignore it and use the previous
    last_start, last_end = zero_runs[-1]
    ends_with_zero_run = (last_end == n - 1)

    if ends_with_zero_run and len(zero_runs) >= 2:
        use_start, use_end = zero_runs[-2]
    else:
        use_start, use_end = last_start, last_end

    # First non-zero AFTER the chosen zero-run
    k = use_end + 1
    while k < n and sales[k] == 0:
        k += 1
    return months[k] if k < n else None


df_wide["SKURestockDate"] = df_wide.apply(restock_after_prev_zero_run, axis=1)

dict_skurestockdate = df_wide["SKURestockDate"].to_dict()

df_wide

Year-Month,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,...,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,SKURestockDate
Offer SKU,,,,,,,,,,,,,,,,,,,,,
0000072400,0,0,0,0,0,0,0,0,0,0,...,8,0,0,0,4,3,5,3,0,2025-04
0000072500,0,0,0,0,0,0,0,0,0,0,...,15,0,0,0,2,1,2,0,1,2025-04
0000072600,0,0,0,2,2,3,0,0,2,8,...,29,1,0,0,1,2,0,0,2,2024-10
0000072800,0,0,0,1,1,0,0,0,0,0,...,7,3,0,0,0,0,1,0,1,2025-06
0000073000,0,0,0,0,0,0,0,0,0,1,...,2,4,2,1,0,0,1,0,0,2024-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0005825900,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,2025-07
0005826000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2025-08
0005826100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2025-07


In [617]:
import re
import pandas as pd

# If Offer SKU is the index, bring it out as a column
if df_wide.index.name == "Offer SKU" or "Offer SKU" not in df_wide.columns:
    df_wide = df_wide.reset_index()

# Identify Year-Month columns like 2023-02, 2025-08, etc.
month_cols = [c for c in df_wide.columns if re.fullmatch(r"\d{4}-\d{2}", str(c))]

# Sort months chronologically
month_cols = sorted(month_cols)

# Everything else (non-months, excluding Offer SKU)
other_cols = [c for c in df_wide.columns if c not in (["Offer SKU"] + month_cols)]

# Reorder: Offer SKU → months → others (e.g., RestockDate)
df_wide = df_wide[["Offer SKU"] + month_cols + other_cols]

# Ensure columns are flat (not MultiIndex)
df_wide.columns = df_wide.columns.astype(str)

# Optional: if you want RestockDate specifically at the very end
if "RestockDate" in df_wide.columns:
    cols = [c for c in df_wide.columns if c not in ["RestockDate"]] + ["RestockDate"]
    df_wide = df_wide[cols]

df_wide

Year-Month,Offer SKU,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,...,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,SKURestockDate
0,0000072400,0,0,0,0,0,0,0,0,0,...,8,0,0,0,4,3,5,3,0,2025-04
1,0000072500,0,0,0,0,0,0,0,0,0,...,15,0,0,0,2,1,2,0,1,2025-04
2,0000072600,0,0,0,2,2,3,0,0,2,...,29,1,0,0,1,2,0,0,2,2024-10
3,0000072800,0,0,0,1,1,0,0,0,0,...,7,3,0,0,0,0,1,0,1,2025-06
4,0000073000,0,0,0,0,0,0,0,0,0,...,2,4,2,1,0,0,1,0,0,2024-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633,0005825900,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,2025-07
3634,0005826000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2025-08
3635,0005826100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2025-07
3636,0005826200,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2025-07


In [656]:
df_wide[df_wide['Offer SKU'] == '0004372500'].to_csv('Temp.csv', index=False)

In [619]:
prices = pd.read_csv('offers (49).csv', delimiter=';', dtype=str)
prices['Discount price'] = (prices['Discount price'].astype(float) * 0.75).round(2)
prices['Sale Price'] = prices['Discount price']
prices['RRP'] = prices['Original price']
prices = prices[['Offer SKU', 'RRP', 'Sale Price']]
prices

,Offer SKU,RRP,Sale Price
0,0005261211,359.00,198.75
1,0004949505,229.00,158.79
2,0004386401,229.00,114.59
3,0004929700,129.00,65.00
4,0004131300,120.00,44.99
...,...,...,...
5524,0005821200,95.00,NaN
5525,0004155802,159.00,110.00
5526,0004155806,159.00,110.00
5527,0005432500,120.00,42.99


In [620]:
vs_prices = pd.read_csv('vs_export_20250722-110248_leather-company.csv', dtype=str)
vs_prices

,VS Parent ID,VS Child ID,Unnamed: 2,Parent Reference,Child Reference,Unnamed: 5,Parent Product Images,Unnamed: 7
0,74,2540,NaN,00014373,0001437300,NaN,https://www.leathercompany.co.uk/images/produc...,NaN
1,74,459,NaN,00014373,0001437301,NaN,https://www.leathercompany.co.uk/images/produc...,NaN
2,74,460,NaN,00014373,0001437302,NaN,https://www.leathercompany.co.uk/images/produc...,NaN
3,74,461,NaN,00014373,0001437303,NaN,https://www.leathercompany.co.uk/images/produc...,NaN
4,74,462,NaN,00014373,0001437304,NaN,https://www.leathercompany.co.uk/images/produc...,NaN
...,...,...,...,...,...,...,...,...
12854,3144,11325,NaN,00034684,0003468404,NaN,https://www.leathercompany.co.uk/images/produc...,NaN
12855,3144,11326,NaN,00034684,0003468405,NaN,https://www.leathercompany.co.uk/images/produc...,NaN
12856,3144,11327,NaN,00034684,0003468406,NaN,https://www.leathercompany.co.uk/images/produc...,NaN
12857,3144,11328,NaN,00034684,0003468407,NaN,https://www.leathercompany.co.uk/images/produc...,NaN


In [621]:
order_prices = df.groupby(['Offer SKU'])['Amount'].mean().reset_index()
order_prices['Amount'] = order_prices['Amount'].round(2)
order_prices['Sale Price'] = order_prices['Amount']

dict_prices = pd.Series(order_prices.Amount.values,index=order_prices['Offer SKU']).to_dict()
prices

,Offer SKU,RRP,Sale Price
0,0005261211,359.00,198.75
1,0004949505,229.00,158.79
2,0004386401,229.00,114.59
3,0004929700,129.00,65.00
4,0004131300,120.00,44.99
...,...,...,...
5524,0005821200,95.00,NaN
5525,0004155802,159.00,110.00
5526,0004155806,159.00,110.00
5527,0005432500,120.00,42.99


In [622]:
df = df[['date', 'Offer SKU', 'Quantity', 'Category Label']]

In [623]:
df['Offer SKU'] = df['Offer SKU'].replace(['0004925700', '0004928100', '0004928200', '0004928800', '0004555700', '0004933600'], ['0005346800', '0005347000', '0005346900', '0005347100', '0005310300', '0005347200'])

In [624]:
fe = pd.read_csv('Fashione EAN 21-07-25.csv', dtype=str)
fe['Offer SKU'] = fe['PLU']
fe = fe[['Style', 'Colour', 'Size', 'Offer SKU']]
fe.head(2)

,Style,Colour,Size,Offer SKU
0,007,BLK/BLUE,S,0002875800
1,007,BLK/BLUE,M,0002875801


In [625]:
df = pd.merge(df, fe, on='Offer SKU', how='left')
# dict_label_style = pd.Series(df.dropna()['Category Label'].values,index=df.dropna()['Style']).to_dict()
# df['Category Label'] = df['Style']
# df['Category Label'] = df['Category Label'].map(dict_label_style)
df.head(2)

,date,Offer SKU,Quantity,Category Label,Style,Colour,Size
0,2023-02-06,0004524800,1,Womens Bags & Purses,s4,DENIM BLUE,NaN
1,2023-02-10,0004525400,1,Womens Bags & Purses,NaN,NaN,NaN


In [626]:
df = df[df['Category Label'] == 'Womens Bags & Purses']

In [627]:
best_sellers = df.groupby(['Style'])['Quantity'].sum().reset_index().sort_values(by='Quantity', ascending=False)['Style'].values
temp = pd.DataFrame(best_sellers, columns=['sellers'])
temp['index1'] = range(1, len(temp)+1)
dict_customs_model = pd.Series(temp.index1.values,index=temp.sellers).to_dict()

In [628]:
q3_2024 = df[(df['date'] >= '2024-07-01') & (df['date'] <= '2024-09-30')]
q4_2024 = df[(df['date'] >= '2024-10-01') & (df['date'] <= '2024-12-31')]
q1_2025 = df[(df['date'] >= '2025-01-01') & (df['date'] <= '2025-03-31')]
q2_2025 = df[(df['date'] >= '2025-04-01') & (df['date'] <= '2025-06-30')]

In [629]:
df['Index'] = df['Style']
df['Index'] = df['Index'].map(dict_customs_model)
df = df.groupby(['Index', 'Style', 'Colour', 'Offer SKU'])['Quantity'].sum().reset_index()

q3_2024['Index'] = q3_2024['Style']
q3_2024['Index'] = q3_2024['Index'].map(dict_customs_model)
q3_2024 = q3_2024.groupby(['Index', 'Style', 'Colour', 'Offer SKU'])['Quantity'].sum().reset_index()
q3_2024['Q3 2024'] = q3_2024['Quantity']
q3_2024 = q3_2024[['Offer SKU', 'Q3 2024']]

df = pd.merge(df, q3_2024, on='Offer SKU', how='left')
df['Q3 2024'] = df['Q3 2024'].replace(np.NaN, 0)
df['Q3 2024'] = df['Q3 2024'].astype(int)

q4_2024['Index'] = q4_2024['Style']
q4_2024['Index'] = q4_2024['Index'].map(dict_customs_model)
q4_2024 = q4_2024.groupby(['Index', 'Style', 'Colour', 'Offer SKU'])['Quantity'].sum().reset_index()
q4_2024['Q4 2024'] = q4_2024['Quantity']
q4_2024 = q4_2024[['Offer SKU', 'Q4 2024']]

df = pd.merge(df, q4_2024, on='Offer SKU', how='left')
df['Q4 2024'] = df['Q4 2024'].replace(np.NaN, 0)
df['Q4 2024'] = df['Q4 2024'].astype(int)

q1_2025['Index'] = q1_2025['Style']
q1_2025['Index'] = q1_2025['Index'].map(dict_customs_model)
q1_2025 = q1_2025.groupby(['Index', 'Style', 'Colour', 'Offer SKU'])['Quantity'].sum().reset_index()
q1_2025['Q1 2025'] = q1_2025['Quantity']
q1_2025 = q1_2025[['Offer SKU', 'Q1 2025']]

df = pd.merge(df, q1_2025, on='Offer SKU', how='left')
df['Q1 2025'] = df['Q1 2025'].replace(np.NaN, 0)
df['Q1 2025'] = df['Q1 2025'].astype(int)

q2_2025['Index'] = q2_2025['Style']
q2_2025['Index'] = q2_2025['Index'].map(dict_customs_model)
q2_2025 = q2_2025.groupby(['Index', 'Style', 'Colour', 'Offer SKU'])['Quantity'].sum().reset_index()
q2_2025['Q2 2025'] = q2_2025['Quantity']
q2_2025 = q2_2025[['Offer SKU', 'Q2 2025']]

df = pd.merge(df, q2_2025, on='Offer SKU', how='left')
df['Q2 2025'] = df['Q2 2025'].replace(np.NaN, 0)
df['Q2 2025'] = df['Q2 2025'].astype(int)

In [630]:
# df['Q1 2025'] = (df['Q4 2024'] * 0.535).round(0).astype(int)
# df['Q2 2025'] = (df['Q4 2024'] * 0.4).round(0).astype(int)
# df['Q3 2025'] = (df['Q4 2024'] * 0.35).round(0).astype(int)

In [631]:
stock = pd.read_csv('stock_levels(38).csv', dtype=str)
stock['Offer SKU'] = stock['reference']
stock['Stock'] = stock['stock'].astype(int)
stock['Stock'].replace(np.NaN, 0, inplace=True)
stock = stock[['Offer SKU', 'Stock']]

df = pd.merge(df, stock, on='Offer SKU', how='left')

df.rename(columns={'Quantity': 'Total sold'}, inplace=True)
# df['Stock'] = df['Stock'].replace(np.NaN, 0, inplace=True)
# df['Stock'] = df['Stock'].astype(int)

In [632]:
### Stock Sort

most_stock = df.groupby(['Style'])['Stock'].sum().reset_index().sort_values(by='Stock', ascending=False)['Style'].values
temp = pd.DataFrame(most_stock, columns=['sellers'])
temp['index1'] = range(1, len(temp)+1)
dict_customs_model = pd.Series(temp.index1.values,index=temp.sellers).to_dict()

df['Index'] = df['Style']
df['Index'] = df['Index'].map(dict_customs_model)

In [633]:
df['Sale Price'] = df['Offer SKU']
df['Stock'] = df['Stock'].fillna(0).astype(int)
df['Sale Price'] = df['Sale Price'].map(dict_prices)

In [634]:
catalog = pd.read_excel('export-products-20250725164338.xlsx', dtype=str, skiprows=1)
catalog['Offer SKU'] = catalog['product_id']
catalog = catalog[~catalog['Offer SKU'].isin(df.reset_index()['Offer SKU'].values)]
catalog = pd.merge(catalog, stock, on='Offer SKU', how='left')
catalog = catalog[catalog['Stock'] != 0]
catalog.dropna(subset=['Stock'], inplace=True)
catalog = pd.merge(catalog, fe, on='Offer SKU', how='left')
catalog.shape

c:\Users\retai\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(158, 53)

In [635]:
df

,Index,Style,Colour,Offer SKU,Total sold,Q3 2024,Q4 2024,Q1 2025,Q2 2025,Stock,Sale Price
0,4,64033,2 tone tan,0004932900,1468,43,125,179,200,240,28.65
1,4,64033,BLACK,0005310300,2783,0,635,401,208,274,28.21
2,4,64033,COGNAC,0004555900,109,0,0,0,0,0,25.22
3,4,64033,GLOME BLUE,0004556100,112,0,0,0,0,0,24.99
4,4,64033,ICE GREY,0004933200,325,28,17,20,22,124,28.74
...,...,...,...,...,...,...,...,...,...,...,...
1171,273,v-83,BROWN,0005819100,1,0,0,0,0,0,75.00
1172,254,v-87,BLACK,0005819600,1,0,0,0,0,1,112.50
1173,228,VIN-7,YELLOW,0004317400,1,0,0,1,0,8,57.99
1174,203,d-99,BLACK,0005023600,1,0,0,0,0,12,32.99


In [636]:
catalog['Index'] = '-'
catalog['Style'] = catalog['parent_product_id']
catalog[['Total sold', 'Q3 2024', 'Q4 2024', 'Q1 2025', 'Q2 2025']] = 0
catalog['Sale Price'] = 0
catalog = catalog[['Index', 'Style', 'Colour', 'Offer SKU', 'Total sold', 'Q3 2024', 'Q4 2024', 'Q1 2025', 'Q2 2025', 'Stock', 'Sale Price']]

df = pd.concat([df, catalog], ignore_index=True)

In [637]:
best_sellers = df.groupby(['Style'])['Total sold'].sum().reset_index().sort_values(by='Total sold', ascending=False)['Style'].values
temp = pd.DataFrame(best_sellers, columns=['sellers'])
temp['index1'] = range(1, len(temp)+1)
dict_customs_model = pd.Series(temp.index1.values,index=temp.sellers).to_dict()

In [638]:
df['Index'] = df['Style']
df['Index'] = df['Index'].map(dict_customs_model)

In [639]:
images = pd.read_csv('vs_export_20250722-110248_leather-company.csv', dtype=str)
images['Offer SKU'] = images['Child Reference']
temp_stock_df = images['Parent Product Images'].str.split('\n', expand=True)
images['Parent Product Images'] = temp_stock_df[0]
images['VS Image'] = images['Parent Product Images']
images['Image'] = images['Parent Product Images']
images = images[['Offer SKU', 'Image']]

df = pd.merge(df, images, on='Offer SKU', how='left')

dict_images = pd.Series(df['Image'].values,index=df['Style']).to_dict()
df['Image'] = df['Style']
df['Image'] = df['Image'].map(dict_images)

In [640]:
df = df.groupby(['Index', 'Image', 'Style', 'Colour', 'Offer SKU', 'Sale Price'])[['Stock', 'Q3 2024', 'Q4 2024', 'Q1 2025', 'Q2 2025']].sum()
df['Total'] = df['Q3 2024'] + df['Q4 2024'] + df['Q1 2025'] + df['Q2 2025']
# df = df[df['Total'] != 0]

In [641]:
df.reset_index(inplace=True)
df['Sale Price'] = df['Sale Price'].replace(0, np.NaN)

temp1 = pd.read_csv('offers (49).csv', delimiter=';', dtype=str)
temp1['Discount price'] = temp1['Discount price'].fillna(temp1['Original price'])
temp1['Discount price'] = (temp1['Discount price'].astype(float) * 0.75).round(2)
temp1 = temp1[['Offer SKU', 'Original price', 'Discount price']]

df = pd.merge(df, temp1, on='Offer SKU', how='left')
df

,Index,Image,Style,Colour,Offer SKU,Sale Price,Stock,Q3 2024,Q4 2024,Q1 2025,Q2 2025,Total,Original price,Discount price
0,1,https://www.leathercompany.co.uk/images/produc...,64033,2 tone tan,0004932900,28.65,240.0,43,125,179,200,547,59.99,29.99
1,1,https://www.leathercompany.co.uk/images/produc...,64033,BLACK,0005310300,28.21,274.0,0,635,401,208,1244,59.99,29.99
2,1,https://www.leathercompany.co.uk/images/produc...,64033,COGNAC,0004555900,25.22,0.0,0,0,0,0,0,NaN,NaN
3,1,https://www.leathercompany.co.uk/images/produc...,64033,GLOME BLUE,0004556100,24.99,0.0,0,0,0,0,0,NaN,NaN
4,1,https://www.leathercompany.co.uk/images/produc...,64033,ICE GREY,0004933200,28.74,124.0,28,17,20,22,87,59.99,29.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,318,https://www.leathercompany.co.uk/images/produc...,v-81,BROWN,0005818700,NaN,1.0,0,0,0,0,0,80.00,60.00
1203,319,https://www.leathercompany.co.uk/images/produc...,v-82,BLACK,0005818800,NaN,1.0,0,0,0,0,0,75.00,56.25
1204,319,https://www.leathercompany.co.uk/images/produc...,v-82,BROWN,0005818900,NaN,1.0,0,0,0,0,0,75.00,56.25
1205,320,https://www.leathercompany.co.uk/images/produc...,v-90,BLACK,0005819800,NaN,1.0,0,0,0,0,0,40.00,30.00


In [642]:
df['RRP'] = df['Original price']
# df['Sale Price'] = df['Sale Price'].fillna(df['Discount price'])
df['Sale Price'] = df['Discount price']
df['Sale Price'] = df['Sale Price'].replace(np.NaN, '-')

In [643]:
df['Cost Price'] = df['Style']
df['Cost Price'] = df['Cost Price'].map(cost_price_dict)
df['Cost Price'] = df['Cost Price'].fillna('-')

In [644]:
df['SKURestockDate'] = df['Offer SKU']
df['SKURestockDate'] = df['SKURestockDate'].map(dict_skurestockdate)

In [645]:
# Ensure SKURestockDate is a datetime type for proper max comparison
df["SKURestockDate"] = pd.to_datetime(df["SKURestockDate"], format="%Y-%m", errors="coerce")

# Group by Style and get the latest restock date
style_restock = df.groupby("Style")["SKURestockDate"].min().reset_index()
style_restock.rename(columns={"SKURestockDate": "StyleRestockDate"}, inplace=True)

# Merge back into df so each row has both SKU restock and style restock
df = df.merge(style_restock, on="Style", how="left")

In [646]:
df

,Index,Image,Style,Colour,Offer SKU,Sale Price,Stock,Q3 2024,Q4 2024,Q1 2025,Q2 2025,Total,Original price,Discount price,RRP,Cost Price,SKURestockDate,StyleRestockDate
0,1,https://www.leathercompany.co.uk/images/produc...,64033,2 tone tan,0004932900,29.99,240.0,43,125,179,200,547,59.99,29.99,59.99,6.95,2023-11-01,2023-11-01
1,1,https://www.leathercompany.co.uk/images/produc...,64033,BLACK,0005310300,29.99,274.0,0,635,401,208,1244,59.99,29.99,59.99,6.95,2024-10-01,2023-11-01
2,1,https://www.leathercompany.co.uk/images/produc...,64033,COGNAC,0004555900,-,0.0,0,0,0,0,0,NaN,NaN,NaN,6.95,NaT,2023-11-01
3,1,https://www.leathercompany.co.uk/images/produc...,64033,GLOME BLUE,0004556100,-,0.0,0,0,0,0,0,NaN,NaN,NaN,6.95,NaT,2023-11-01
4,1,https://www.leathercompany.co.uk/images/produc...,64033,ICE GREY,0004933200,29.99,124.0,28,17,20,22,87,59.99,29.99,59.99,6.95,2023-11-01,2023-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,318,https://www.leathercompany.co.uk/images/produc...,v-81,BROWN,0005818700,60.0,1.0,0,0,0,0,0,80.00,60.00,80.00,-,NaT,NaT
1203,319,https://www.leathercompany.co.uk/images/produc...,v-82,BLACK,0005818800,56.25,1.0,0,0,0,0,0,75.00,56.25,75.00,-,NaT,NaT
1204,319,https://www.leathercompany.co.uk/images/produc...,v-82,BROWN,0005818900,56.25,1.0,0,0,0,0,0,75.00,56.25,75.00,-,NaT,NaT
1205,320,https://www.leathercompany.co.uk/images/produc...,v-90,BLACK,0005819800,30.0,1.0,0,0,0,0,0,40.00,30.00,40.00,-,NaT,NaT


In [647]:
df = df.groupby(['Index', 'Image', 'Style', 'Colour', 'Offer SKU', 'SKURestockDate', 'StyleRestockDate', 'Cost Price', 'RRP', 'Sale Price'])[['Stock', 'Q3 2024', 'Q4 2024', 'Q1 2025', 'Q2 2025']].sum()
df['Total Sold'] = df['Q3 2024'] + df['Q4 2024'] + df['Q1 2025'] + df['Q2 2025']

In [648]:
# df.to_excel('Handbags July 2025.xlsx')
df.reset_index().to_csv('Handbags July 2025 No Groups.csv', index=False)

In [649]:
df.reset_index().isnull().sum()

Index               0
Image               0
Style               0
Colour              0
Offer SKU           0
SKURestockDate      0
StyleRestockDate    0
Cost Price          0
RRP                 0
Sale Price          0
Stock               0
Q3 2024             0
Q4 2024             0
Q1 2025             0
Q2 2025             0
Total Sold          0
dtype: int64

In [650]:
temp = df.reset_index().copy()

In [651]:
temp['Style-CostPrice'] = temp['Style'] + '-' + temp['Sale Price'].astype(str)
temp['Style-CostPrice'].value_counts()

Style-CostPrice
63753-39.99        17
63014/cb3-29.99    17
cb-5-29.99         16
64033-29.99        11
x-30-24.99         10
                   ..
52234-119.25        1
D-73-65.0           1
62833-54.99         1
20-186-39.99        1
v-87-112.5          1
Name: count, Length: 261, dtype: int64

In [652]:
temp

,Index,Image,Style,Colour,Offer SKU,SKURestockDate,StyleRestockDate,Cost Price,RRP,Sale Price,Stock,Q3 2024,Q4 2024,Q1 2025,Q2 2025,Total Sold,Style-CostPrice
0,1,https://www.leathercompany.co.uk/images/produc...,64033,2 tone tan,0004932900,2023-11-01,2023-11-01,6.95,59.99,29.99,240.0,43,125,179,200,547,64033-29.99
1,1,https://www.leathercompany.co.uk/images/produc...,64033,BLACK,0005310300,2024-10-01,2023-11-01,6.95,59.99,29.99,274.0,0,635,401,208,1244,64033-29.99
2,1,https://www.leathercompany.co.uk/images/produc...,64033,ICE GREY,0004933200,2023-11-01,2023-11-01,6.95,59.99,29.99,124.0,28,17,20,22,87,64033-29.99
3,1,https://www.leathercompany.co.uk/images/produc...,64033,RASPBERRY,0005347200,2024-11-01,2023-11-01,6.95,59.99,29.99,191.0,0,15,20,17,52,64033-29.99
4,1,https://www.leathercompany.co.uk/images/produc...,64033,ROSE,0004933700,2024-10-01,2023-11-01,6.95,59.99,29.99,202.0,0,31,30,28,89,64033-29.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,292,https://www.leathercompany.co.uk/images/produc...,62093,FUCHSIA,0003747400,2025-05-01,2025-05-01,-,59.00,44.25,19.0,0,0,0,1,1,62093-44.25
989,293,https://www.leathercompany.co.uk/images/produc...,v-83,BROWN,0005819100,2025-07-01,2025-07-01,-,100.00,75.0,0.0,0,0,0,0,0,v-83-75.0
990,294,https://www.leathercompany.co.uk/images/produc...,VIN-7,YELLOW,0004317400,2025-02-01,2025-02-01,-,89.95,57.99,8.0,0,0,1,0,1,VIN-7-57.99
991,295,https://www.leathercompany.co.uk/images/produc...,64741,BEIGE,0005441700,2025-07-01,2025-07-01,10.55,110.00,80.0,74.0,0,0,0,0,0,64741-80.0


In [653]:
import pandas as pd

# Example dataset with all edge cases
data = {
    "2024-01": [5, 0, 1, 3, 0, 0],
    "2024-02": [3, 0, 1, 2, 0, 0],
    "2024-03": [0, 0, 1, 0, 0, 0],
    "2024-04": [0, 0, 1, 0, 0, 0],
    "2024-05": [2, 0, 1, 5, 0, 0],
    "2024-06": [4, 0, 1, 8, 0, 0],
}

df = pd.DataFrame(data, index=[
    "ProductA",  # restock after two zeros (expect 2024-05)
    "ProductB",  # all zeros (never in stock → None)
    "ProductC",  # always in stock (no consecutive zeros → 2024-01)
    "ProductD",  # two zeros then restock (expect 2024-05)
    "ProductE",  # all zeros (never in stock → None)
    "ProductF",  # all zeros (never in stock → None)
])

print(df)

          2024-01  2024-02  2024-03  2024-04  2024-05  2024-06
ProductA        5        3        0        0        2        4
ProductB        0        0        0        0        0        0
ProductC        1        1        1        1        1        1
ProductD        3        2        0        0        5        8
ProductE        0        0        0        0        0        0
ProductF        0        0        0        0        0        0


In [654]:
def get_in_stock_month(row):
    sales = row.values[::-1]   # latest → oldest
    months = row.index[::-1]

    for i in range(len(sales) - 1):
        if sales[i] == 0 and sales[i+1] == 0:   # found 2 consecutive zeros
            if i > 0 and sales[i-1] > 0:        # check non-zero before zeros
                return months[i-1]
            break
    
    # If no consecutive zeros → always in stock
    return months[-1]   # oldest month


df["CameIntoStock"] = df.apply(get_in_stock_month, axis=1)
print(df)

          2024-01  2024-02  2024-03  2024-04  2024-05  2024-06 CameIntoStock
ProductA        5        3        0        0        2        4       2024-05
ProductB        0        0        0        0        0        0       2024-01
ProductC        1        1        1        1        1        1       2024-01
ProductD        3        2        0        0        5        8       2024-05
ProductE        0        0        0        0        0        0       2024-01
ProductF        0        0        0        0        0        0       2024-01


In [696]:
temp = pd.read_csv('vs_export_20250819-120850_leather-company.csv', dtype=str)
# temp['Child Reference'] = temp['Child Reference'].str.zfill(10)
temp.head(2)

,VS Parent ID,VS Child ID,Unnamed: 2,Parent Reference,Child Reference,Unnamed: 5,Parent Product Title,Child Product Title,Product Subtitle,Product Description,...,EAN,Unnamed: 19,Price (Inc VAT),Unnamed: 21,Stock Value,Pick Location,Unnamed: 24,Parent Product Images,Child Product Images,Unnamed: 27
0,74,2540,NaN,00014373,0001437300,NaN,Leather Jacket Black/ani : Juliet,Leather Jacket Black/ani : Juliet,Black/ani,"<p><span style=""color: #808080; font-size: 14p...",...,NaN,NaN,249.000,NaN,0,NaN,NaN,https://www.leathercompany.co.uk/images/produc...,NaN,NaN
1,74,459,NaN,00014373,0001437301,NaN,Leather Jacket Black/ani : Juliet,Leather Jacket Black/ani : Juliet,Black/ani,"<p><span style=""color: #808080; font-size: 14p...",...,5056798412148,NaN,249.000,NaN,8,NaN,NaN,https://www.leathercompany.co.uk/images/produc...,NaN,NaN


In [697]:
t2 = temp['Parent Product Images'].str.split('\n', expand=True)
temp['Parent Product Images'] = t2[0]

In [698]:
template = pd.read_csv('ProductImportSample.csv', dtype=str)
template = template.iloc[0:0]
template.head()

,SKU,Name,WarehouseLocation,Weight,Category,Tag1,Tag2,Tag3,Tag4,Tag5,...,ThumbnailUrl,UPC,FillSKU,Length,Width,Height,UseProductName,Active,ParentSKU,IsReturnable


In [699]:
fe = pd.read_csv('Fashione EAN 21-07-25.csv', dtype=str)
fe['Child Reference'] = fe['PLU']
fe['Size'] = fe['Size'].fillna('ONE SIZE')
fe = fe[['Child Reference', 'Style', 'Colour', 'Size']]

temp = pd.merge(temp, fe, on='Child Reference', how='left')

In [700]:
category = pd.read_csv('Category-Department 28-07-25.csv', skiprows=2, dtype=str)
category = category[~category['Style'].isna()]
category = category[category['Style'] != 'Style']
category = category[~category['Category'].isna()]
category = category[['Style', 'Category']]

temp = pd.merge(temp, category, on='Style', how='left')

In [ ]:
template['SKU'] = temp['Child Reference']
template['Name'] = temp['Model Number'] + ' | ' + temp['Colour'] + ' | ' + temp['Size']
template['WarehouseLocation'] = temp['Pick Location']
template['Category'] = temp['Category']
template['ThumbnailUrl'] = temp['Parent Product Images']
template['UPC'] = temp['EAN']
template['Active'] = 'True'
template['IsReturnable'] = 'TRUE'
template['ParentSKU'] = temp['Parent Reference']

In [ ]:
template[template['ParentSKU'] == '00014373'].drop(['Parent SKU'], axis=1).to_csv('Test ShipStation.csv', index=False)

: 